In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd
from pathlib import Path
import pyarrow.parquet as pq

month = 1
year = 2023
path = Path("..") / "data" / "processed" / f"citi_rides_{year}_{month:02}.parquet"

table = pq.read_table(path)
rides = table.to_pandas()
rides.iloc[1000:1020]

,started_at,start_station_id
1000,2023-01-30 20:30:29.844,6912.01
1001,2023-01-20 16:30:02.483,7049.04
1002,2023-01-14 19:56:45.189,6912.01
1003,2023-01-11 23:32:53.354,4659.02
1004,2023-01-26 12:58:21.715,7049.04
1005,2023-01-31 16:33:44.798,7602.05
1006,2023-01-17 18:53:30.310,6526.01
1007,2023-01-19 08:38:10.259,7602.05
1008,2023-01-05 15:39:29.256,7407.13
1009,2023-01-11 15:30:23.852,7407.13


In [3]:
rides = rides.sort_values(by='started_at', ascending=True).reset_index(drop=True)

In [4]:
rides.head()

,started_at,start_station_id
0,2023-01-01 00:00:13.021,6955.01
1,2023-01-01 00:00:15.653,7634.01
2,2023-01-01 00:00:27.436,4307.13
3,2023-01-01 00:00:29.074,8528.05
4,2023-01-01 00:00:49.245,7340.07


In [5]:
rides.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1777006 entries, 0 to 1777005
Data columns (total 2 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   started_at        object
 1   start_station_id  object
dtypes: object(2)
memory usage: 27.1+ MB


In [6]:
rides['started_at'] = pd.to_datetime(rides['started_at'])

In [7]:
rides.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1777006 entries, 0 to 1777005
Data columns (total 2 columns):
 #   Column            Dtype         
---  ------            -----         
 0   started_at        datetime64[ns]
 1   start_station_id  object        
dtypes: datetime64[ns](1), object(1)
memory usage: 27.1+ MB


In [8]:
rides['start_hour'] = rides['started_at'].dt.floor('h')
rides.tail()

,started_at,start_station_id,start_hour
1777001,2023-01-31 23:54:04.004,6345.08,2023-01-31 23:00:00
1777002,2023-01-31 23:54:12.671,5294.04,2023-01-31 23:00:00
1777003,2023-01-31 23:54:13.263,4895.09,2023-01-31 23:00:00
1777004,2023-01-31 23:55:27.739,4743.04,2023-01-31 23:00:00
1777005,2023-01-31 23:57:54.025,4446.05,2023-01-31 23:00:00


In [9]:
top_start_stations = rides['start_station_id'].value_counts().head(3)
print('Top 3 Start Stations:\n', top_start_stations)

Top 3 Start Stations:
 start_station_id
6140.05    8549
5905.14    6907
6822.09    6877
Name: count, dtype: int64


In [10]:
rides = rides[rides['start_station_id'].isin(['6140.05', '5905.14', '6822.09'])]
rides

,started_at,start_station_id,start_hour
100,2023-01-01 00:10:42.722,6140.05,2023-01-01 00:00:00
367,2023-01-01 00:27:40.052,6822.09,2023-01-01 00:00:00
598,2023-01-01 00:39:13.016,6822.09,2023-01-01 00:00:00
825,2023-01-01 00:51:27.940,6822.09,2023-01-01 00:00:00
1050,2023-01-01 01:01:10.818,6140.05,2023-01-01 01:00:00
...,...,...,...
1776572,2023-01-31 23:14:01.842,6822.09,2023-01-31 23:00:00
1776663,2023-01-31 23:20:41.036,5905.14,2023-01-31 23:00:00
1776835,2023-01-31 23:35:48.691,5905.14,2023-01-31 23:00:00
1776977,2023-01-31 23:50:00.792,5905.14,2023-01-31 23:00:00


In [11]:
agg_rides = rides.groupby(['start_hour', 'start_station_id']).size().reset_index()
agg_rides.rename(columns={0: 'rides'}, inplace=True)
agg_rides.iloc[1000:1020]

,start_hour,start_station_id,rides
1000,2023-01-17 06:00:00,6822.09,14
1001,2023-01-17 07:00:00,5905.14,10
1002,2023-01-17 07:00:00,6140.05,12
1003,2023-01-17 07:00:00,6822.09,16
1004,2023-01-17 08:00:00,5905.14,17
1005,2023-01-17 08:00:00,6140.05,25
1006,2023-01-17 08:00:00,6822.09,26
1007,2023-01-17 09:00:00,5905.14,13
1008,2023-01-17 09:00:00,6140.05,21
1009,2023-01-17 09:00:00,6822.09,20


In [12]:
def fill_missing_rides_full_range(df, hour_col, location_col, rides_col):
    """
    Fills in missing rides for all hours in the range and all unique locations.

    Parameters:
    - df: DataFrame with columns [hour_col, location_col, rides_col]
    - hour_col: Name of the column containing hourly timestamps
    - location_col: Name of the column containing location IDs
    - rides_col: Name of the column containing ride counts

    Returns:
    - DataFrame with missing hours and locations filled in with 0 rides
    """
    # Ensure the hour column is in datetime format
    df[hour_col] = pd.to_datetime(df[hour_col])

    # Get the full range of hours (from min to max) with hourly frequency
    full_hours = pd.date_range(
        start=df[hour_col].min(),
        end=df[hour_col].max(),
        freq="h"
    )

    # Get all unique location IDs
    all_locations = df[location_col].unique()

    # Create a DataFrame with all combinations of hours and locations
    full_combinations = pd.DataFrame(
        [(hour, location) for hour in full_hours for location in all_locations],
        columns=[hour_col, location_col]
    )

    # Merge the original DataFrame with the full combinations DataFrame
    merged_df = pd.merge(full_combinations, df, on=[hour_col, location_col], how='left')

    # Fill missing rides with 0
    merged_df[rides_col] = merged_df[rides_col].fillna(0).astype(int)

    return merged_df

In [13]:
hour_col = "start_hour"
location_col = "start_station_id"
rides_col = "rides"
agg_data_filled = fill_missing_rides_full_range(agg_rides, hour_col, location_col, rides_col).sort_values(["start_station_id", "start_hour"]).reset_index(drop=True)

In [14]:
agg_data_filled.head(15)

,start_hour,start_station_id,rides
0,2023-01-01 00:00:00,5905.14,0
1,2023-01-01 01:00:00,5905.14,5
2,2023-01-01 02:00:00,5905.14,7
3,2023-01-01 03:00:00,5905.14,3
4,2023-01-01 04:00:00,5905.14,2
5,2023-01-01 05:00:00,5905.14,1
6,2023-01-01 06:00:00,5905.14,2
7,2023-01-01 07:00:00,5905.14,0
8,2023-01-01 08:00:00,5905.14,2
9,2023-01-01 09:00:00,5905.14,2


In [15]:
from typing import Optional, List
import plotly.express as px
import pandas as pd

def plot_rides(
    rides: pd.DataFrame,
    locations: Optional[List[int]] = None
):

    rides_to_plot = rides[rides.start_station_id.isin(locations)] if locations else rides

    fig = px.line(
        rides_to_plot,
        x="start_hour",
        y="rides",
        color="start_station_id",
        template="none"
    )

    fig.show()

In [16]:
plot_rides(agg_data_filled, locations=['5905.14', '6140.05', '6822.09'])

In [17]:
path = Path('..') / 'data' / 'processed' / f"ts_data_{year}_{month:02}.parquet"
agg_data_filled.to_parquet(path, engine='pyarrow', index=False)